In [ ]:
# !pip install mxnet
# !pip install gluonnlp pandas tqdm
# !pip install sentencepiece
# !pip install transformers
# !pip install torch

In [ ]:
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [2]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [3]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from sklearn.model_selection import train_test_split
import pandas as pd

In [4]:
##GPU 사용 시
device = torch.device("cuda:0")

In [5]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [ ]:
# !wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
# !wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

In [38]:
# dataset = nlp.data.TSVDataset("fully_labelled_df_2.txt", field_indices=[2,3], num_discard_samples=1)
# # dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [39]:
dataset

In [40]:
raw_dataset = pd.read_csv('fully_labelled_df_2.csv')
raw_dataset = raw_dataset[['text', '구체적 기재']]

In [41]:
raw_dataset = raw_dataset.values.tolist()

In [43]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [9]:
# dataset_train, dataset_test = train_test_split(dataset, test_size=0.25, random_state=0)

In [44]:
# 토큰화, 정수 인코딩, 패딩
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[int(label_idx)]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [27]:
## Setting parameters
max_len = 512
batch_size = 2
warmup_ratio = 0.1
num_epochs = 30
max_grad_norm = 1
log_interval = 200
learning_rate =  2e-5

In [45]:
dataset = BERTDataset(raw_dataset, 0, 1, tok, max_len, True, False)
# data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [46]:
dataset

In [14]:
dataset_train, dataset_test = train_test_split(dataset, test_size=0.2, random_state=0)

In [15]:
dataset_train

[(array([   2,  994, 2801, 5538, 6185, 5760,  887, 7276,  517,   46, 1240,
         6493,  517,   46, 2770, 6493,  517,   46, 3004, 6493,  517,   54,
          771, 7234, 6812, 7086, 1280, 6896, 4595, 7828, 3264,  517,   46,
         1289, 7167, 3923, 7088, 2813, 5760, 3264,  517,   54, 1926, 7407,
         7086, 4297, 7119,  517,   46, 2770, 6493,  517,   46, 1240, 6493,
          517,   54,  994, 5475, 5760, 4213, 5561, 7206, 1936, 7088, 4754,
         3016, 7944, 6364,  517,   46, 2850, 7402, 5495, 3592,  517,   54,
         2896, 7024, 3996, 7227, 7020, 5330, 7095, 1043, 6629, 7276,  517,
           46,  517,  453,  517,   54,  517,  389, 6116, 4754, 3533, 5330,
         7217, 5341,  517,   54, 5016, 7949, 7086, 2030, 5468, 2850,  517,
           46, 3047, 5468, 1289, 6064, 7940,  517,   54, 5016, 7166, 7086,
         4186, 6064,  517,   46, 1289, 6064,  517,   54, 5016, 7682, 7659,
         5691, 5760,  887, 7276,  517,   46, 2770, 6493, 2845,  517,   46,
         1240, 6493,  517

In [16]:
len(dataset_train)

412

In [17]:
len(dataset_test)

104

In [18]:
train_dataloader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, num_workers=5)

In [29]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [30]:
model = BERTClassifier(bertmodel,  dr_rate=0.2).to(device)

In [31]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [32]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [33]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [34]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [35]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [36]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-36-e6a38b13095b>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


epoch 1 batch id 1 loss 1.1340241432189941 train acc 0.5
epoch 1 batch id 201 loss 0.9540058374404907 train acc 0.5049751243781094

epoch 1 train acc 0.5145631067961165


<ipython-input-36-e6a38b13095b>:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):



epoch 1 test acc 0.6153846153846154


epoch 2 batch id 1 loss 0.49726754426956177 train acc 1.0
epoch 2 batch id 201 loss 1.041081428527832 train acc 0.7288557213930348

epoch 2 train acc 0.7330097087378641


...


epoch 28 test acc 0.6057692307692307


epoch 29 batch id 1 loss 0.0016405582427978516 train acc 1.0
epoch 29 batch id 201 loss 0.003023386001586914 train acc 0.9776119402985075

epoch 29 train acc 0.9781553398058253



epoch 29 test acc 0.6057692307692307


epoch 30 batch id 1 loss 0.0018815994262695312 train acc 1.0
epoch 30 batch id 201 loss 0.0032367706298828125 train acc 0.9751243781094527

epoch 30 train acc 0.9757281553398058



epoch 30 test acc 0.6057692307692307
